In [1]:
import os
import sys
import torch
from PIL import Image

sys.path.append("./")

PRETRAINED = './MODEL/raso_zeroshot.pth'

from raso.models import raso
from raso import inference_ram as inference
from raso import get_transform


def get_model(path, device):
    transform = get_transform(image_size=384)
    #######load model
    model = raso(pretrained=path,
                        image_size=384,
                        vit='swin_l')
    model.eval()
    model = model.to(device)
    return model, transform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, transform = get_model(PRETRAINED, device)

BertLMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://n

/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
./MODEL/raso_zeroshot.pth
--------------
load checkpoint from ./MODEL/raso_zeroshot.pth


In [ ]:
test_image = "./examples/img_01.png"
image_pil = Image.open(test_image)
image = transform(image_pil).unsqueeze(0).to(device)

res = inference(image, model) # using default threshold 0.65
print("Results with default threshold (0.65):", res[0])

# to change a different threshold
model.class_threshold = torch.tensor([0.75] * 2066).to(model.class_threshold.device)
res = inference(image, model) # using threshold 0.75
print("Results with threshold 0.75:", res[0])

Results with default threshold (0.65): bile duct | dissector | gallbladder | grasper | grasping forceps | liver | retractor | laparoscopic grasper | common bile duct | laparoscopic dissector | laparoscopic instrument
Results with threshold 0.75: dissector | gallbladder | grasper | liver | laparoscopic grasper
